In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import sys
sys.path.append("../dataset/original/")
from evaluation_script import *

In [17]:
score_path = '../lgb_predictions_full.csv'
s = pd.read_csv(score_path)

In [3]:
train = pd.read_csv("../dataset/original/train.csv", escapechar="\\")
test = pd.read_csv("../dataset/original/test.csv", escapechar="\\")

In [18]:
s.ordered_scores = [eval(x) for x in s.ordered_scores]
s.ordered_linked = [eval(x) for x in s.ordered_linked]
s.ordered_record = [eval(x) for x in s.ordered_record]

In [21]:
s['top10_scores'] = s.ordered_scores.apply(lambda x: x[:10])
s['top10_linked'] = s.ordered_linked.apply(lambda x: x[:10])
s['top10_record'] = s.ordered_record.apply(lambda x: x[:10])

In [7]:
def expand_df(s):
    # Expand df
    new_df = []
    for (q, l, r, sc) in tqdm(zip(s.queried_record_id, s.ordered_linked, s.ordered_record, s.ordered_scores)):
            for x in range(len(l)):
                new_df.append((q, l[x], r[x], sc[x]))

    sub = pd.DataFrame(new_df, columns=['queried_record_id', 'predicted_linked_id', 'predicted_record_id', 'score'])
    return sub

# Add linked_id to the test

In [10]:
test['linked_id'] = test.record_id.str.split("-")
test['linked_id'] = test.linked_id.apply(lambda x: x[0])
test.linked_id = test.linked_id.astype(int)

# Top10

In [25]:
renaming_dict = {'top10_scores':'ordered_scores', 'top10_linked':'ordered_linked', 'top10_record':'ordered_record'}
s_top10 = s[['queried_record_id', 'top10_scores', 'top10_linked', 'top10_record']].rename(columns=renaming_dict)

In [27]:
sub = expand_df(s_top10)

266955it [00:02, 126111.64it/s]


In [ ]:
precision = precision_at_k(sub[['queried_record_id', 'predicted_record_id']], train.set_index("record_id"), test.set_index("record_id"))

 25%|██▌       | 67650/266955 [00:40<04:38, 715.37it/s] 

In [ ]:
precision['AveragePrecision']

In [13]:
recall = recall_at_k(sub[['queried_record_id', 'predicted_record_id']], train.set_index("record_id"), test.set_index("record_id"))

100%|██████████| 266955/266955 [02:19<00:00, 1909.79it/s]


In [14]:
recall['AverageRecall']

0.9869789294113339

# Restricted

La precision è calcolata in questo modo:
### precision = len(selectedRelevantRecords) / len(setPredictedRecords)
Riducendo il numero di predizioni a quelle 'sicure', dovremmo aumentare la precision stessa. Quello che è stato fatto è questo: supponiamo di avere

| index | queried_record_id |predicted_linked_id | predicted_record_id | score
| --- | --- | --- | --- | --- |
| 0 | 15004001-T0-TST-CP | 15004001 | 15004001-T0 | 7.553092
| 1 | 15004001-T0-TST-CP | 15004001 | 15004001-M0 | 7.553092
| 2 | 15004001-T0-TST-CP |	15004001 |	15004001 |	7.553092
| 3 |	15004001-T0-TST-CP |	15004001 |	15004001-T2 |	7.553092
| 4 |	15004001-T0-TST-CP |	15004001 |	15004001-T1 |	4.449898
| 5 |	15004001-T0-TST-CP |	15004001 |	15004001-T4 |	4.403244
| 6 |	15004001-T0-TST-CP |	15004001 |	15004001-M2 |	1.808847
| 7 |	15004001-T0-TST-CP |	15004001 |	15004001-M1 |	1.808847
| 8 |	15004001-T0-TST-CP |	15004001 |	15004001-NV1 |	0.667029
| 9 |	15004001-T0-TST-CP |	12187297 |	12187297 |	-7.709455

il **delta** corrisponde alla differenza tra lo score più alto e l'i-esimo score: s[0] - s[i]. Per definire una certa 'sicurezza' nelle raccomandazioni poniamo max_delta = 2.0. Dunque la submission ristretta considera, tra i top10, quelli che hanno score distante <= max_delta, inoltre vengono comunque inserite quelle predizioni k tra le top10 che hanno lo stesso linked_id della predizione con score più alto (il primo): linked_id_k == linked_id_0. In questo modo raccomandiamo i record_id di cui siamo 'sicuri'. Nell'esempio, le row 1,2 e 3 vengono mantenute come raccomandazioni poiché hanno lo score che dista <= di max_delta. Inoltre le row 4,5,6,7 e 8 vengono comunque mantenute poiché hanno lo stesso linked_id della row 0. L'unico che viene escluso è la row 9 perché lo score > max_delta e il linked_id non è lo stesso.

In [13]:
def restricted_df(s):
    restricted_pred = []
    max_delta = 2.0
    for (q, sc, rec, l) in tqdm(zip(s.queried_record_id, s.ordered_scores, s.ordered_record, s.ordered_linked)):
        for x in range(len(sc)):
            if x == 0: # Il primo elemento predetto si mette sempre [quello con score più alto]
                restricted_pred.append((q, sc[x], rec[x]))
            else:
                if x >= 10:
                    continue
                elif (sc[0] - sc[x] < max_delta) or (l[0] == l[x]):   # se le predizioni hanno uno scores che dista < max_delta dalla prima allora si inseriscono
                    restricted_pred.append((q, sc[x], rec[x]))
                else:
                    continue
    restricted_df = pd.DataFrame(restricted_pred, columns=['queried_record_id', 'scores', 'predicted_record_id'])
    return restricted_df

In [14]:
restricted_df = restricted_df(s)

In [15]:
precision_rest = precision_at_k(restricted_df[['queried_record_id', 'predicted_record_id']], train.set_index("record_id"), test.set_index("record_id"))

processed 1000/266955 records, 0.37%
	current precision: 0.98
	time elapsed: 12.38 s
processed 2000/266955 records, 0.75%
	current precision: 0.98
	time elapsed: 13.05 s
processed 3000/266955 records, 1.12%
	current precision: 0.98
	time elapsed: 13.76 s
processed 4000/266955 records, 1.50%
	current precision: 0.99
	time elapsed: 14.43 s
processed 5000/266955 records, 1.87%
	current precision: 0.99
	time elapsed: 15.09 s
processed 6000/266955 records, 2.25%
	current precision: 0.99
	time elapsed: 15.73 s
processed 7000/266955 records, 2.62%
	current precision: 0.99
	time elapsed: 16.31 s
processed 8000/266955 records, 3.00%
	current precision: 0.99
	time elapsed: 16.91 s
processed 9000/266955 records, 3.37%
	current precision: 0.99
	time elapsed: 17.50 s
processed 10000/266955 records, 3.75%
	current precision: 0.99
	time elapsed: 18.11 s
processed 11000/266955 records, 4.12%
	current precision: 0.99
	time elapsed: 18.66 s
processed 12000/266955 records, 4.50%
	current precision: 0.99


processed 96000/266955 records, 35.96%
	current precision: 0.99
	time elapsed: 73.40 s
processed 97000/266955 records, 36.34%
	current precision: 0.99
	time elapsed: 74.62 s
processed 98000/266955 records, 36.71%
	current precision: 0.99
	time elapsed: 75.25 s
processed 99000/266955 records, 37.08%
	current precision: 0.99
	time elapsed: 75.99 s
processed 100000/266955 records, 37.46%
	current precision: 0.99
	time elapsed: 76.62 s
processed 101000/266955 records, 37.83%
	current precision: 0.99
	time elapsed: 78.03 s
processed 102000/266955 records, 38.21%
	current precision: 0.99
	time elapsed: 79.25 s
processed 103000/266955 records, 38.58%
	current precision: 0.99
	time elapsed: 80.38 s
processed 104000/266955 records, 38.96%
	current precision: 0.99
	time elapsed: 81.32 s
processed 105000/266955 records, 39.33%
	current precision: 0.99
	time elapsed: 82.25 s
processed 106000/266955 records, 39.71%
	current precision: 0.99
	time elapsed: 82.99 s
processed 107000/266955 records, 40.

processed 189000/266955 records, 70.80%
	current precision: 0.99
	time elapsed: 137.97 s
processed 190000/266955 records, 71.17%
	current precision: 0.99
	time elapsed: 138.59 s
processed 191000/266955 records, 71.55%
	current precision: 0.99
	time elapsed: 139.17 s
processed 192000/266955 records, 71.92%
	current precision: 0.99
	time elapsed: 139.79 s
processed 193000/266955 records, 72.30%
	current precision: 0.99
	time elapsed: 140.37 s
processed 194000/266955 records, 72.67%
	current precision: 0.99
	time elapsed: 140.96 s
processed 195000/266955 records, 73.05%
	current precision: 0.99
	time elapsed: 142.63 s
processed 196000/266955 records, 73.42%
	current precision: 0.99
	time elapsed: 143.23 s
processed 197000/266955 records, 73.80%
	current precision: 0.99
	time elapsed: 143.80 s
processed 198000/266955 records, 74.17%
	current precision: 0.99
	time elapsed: 144.36 s
processed 199000/266955 records, 74.54%
	current precision: 0.99
	time elapsed: 144.97 s
processed 200000/2669

In [16]:
recall_rest = recall_at_k(restricted_df[['queried_record_id', 'predicted_record_id']], train.set_index("record_id"), test.set_index("record_id"))

processed 1000/266955 records, 0.37%
	current recall: 0.98
	time elapsed: 13.83 s
processed 2000/266955 records, 0.75%
	current recall: 0.98
	time elapsed: 14.49 s
processed 3000/266955 records, 1.12%
	current recall: 0.99
	time elapsed: 15.09 s
processed 4000/266955 records, 1.50%
	current recall: 0.99
	time elapsed: 15.69 s
processed 5000/266955 records, 1.87%
	current recall: 0.99
	time elapsed: 16.29 s
processed 6000/266955 records, 2.25%
	current recall: 0.99
	time elapsed: 16.91 s
processed 7000/266955 records, 2.62%
	current recall: 0.99
	time elapsed: 17.55 s
processed 8000/266955 records, 3.00%
	current recall: 0.99
	time elapsed: 18.22 s
processed 9000/266955 records, 3.37%
	current recall: 0.99
	time elapsed: 18.85 s
processed 10000/266955 records, 3.75%
	current recall: 0.99
	time elapsed: 19.43 s
processed 11000/266955 records, 4.12%
	current recall: 0.99
	time elapsed: 20.01 s
processed 12000/266955 records, 4.50%
	current recall: 0.99
	time elapsed: 20.67 s
processed 130

processed 99000/266955 records, 37.08%
	current recall: 0.99
	time elapsed: 73.73 s
processed 100000/266955 records, 37.46%
	current recall: 0.99
	time elapsed: 74.30 s
processed 101000/266955 records, 37.83%
	current recall: 0.99
	time elapsed: 74.95 s
processed 102000/266955 records, 38.21%
	current recall: 0.99
	time elapsed: 75.59 s
processed 103000/266955 records, 38.58%
	current recall: 0.99
	time elapsed: 76.18 s
processed 104000/266955 records, 38.96%
	current recall: 0.99
	time elapsed: 76.80 s
processed 105000/266955 records, 39.33%
	current recall: 0.99
	time elapsed: 77.36 s
processed 106000/266955 records, 39.71%
	current recall: 0.99
	time elapsed: 77.97 s
processed 107000/266955 records, 40.08%
	current recall: 0.99
	time elapsed: 78.59 s
processed 108000/266955 records, 40.46%
	current recall: 0.99
	time elapsed: 79.19 s
processed 109000/266955 records, 40.83%
	current recall: 0.99
	time elapsed: 79.86 s
processed 110000/266955 records, 41.21%
	current recall: 0.99
	tim

processed 195000/266955 records, 73.05%
	current recall: 0.99
	time elapsed: 137.33 s
processed 196000/266955 records, 73.42%
	current recall: 0.99
	time elapsed: 137.96 s
processed 197000/266955 records, 73.80%
	current recall: 0.99
	time elapsed: 138.54 s
processed 198000/266955 records, 74.17%
	current recall: 0.99
	time elapsed: 139.11 s
processed 199000/266955 records, 74.54%
	current recall: 0.99
	time elapsed: 139.67 s
processed 200000/266955 records, 74.92%
	current recall: 0.99
	time elapsed: 140.24 s
processed 201000/266955 records, 75.29%
	current recall: 0.99
	time elapsed: 140.86 s
processed 202000/266955 records, 75.67%
	current recall: 0.99
	time elapsed: 141.45 s
processed 203000/266955 records, 76.04%
	current recall: 0.99
	time elapsed: 142.02 s
processed 204000/266955 records, 76.42%
	current recall: 0.99
	time elapsed: 143.77 s
processed 205000/266955 records, 76.79%
	current recall: 0.99
	time elapsed: 144.41 s
processed 206000/266955 records, 77.17%
	current recal

## AverageFilteredRecord-Precision

In [34]:
queries_with_relevant_records = precision_rest['perQueryResult'][precision_rest['perQueryResult']["AllRelevantRecords"] > 0]

In [35]:
queries_with_relevant_records

,QueriedRecordID,Precision@K,SelectedRecords,AllRelevantRecords
10000010-TST-CP,10000010-TST-CP,1.000000,1,1
10000018-T4-TST-CP,10000018-T4-TST-CP,1.000000,5,5
10000018-TST-M,10000018-TST-M,1.000000,5,5
10000020-NV0-TST-CP,10000020-NV0-TST-CP,1.000000,3,3
10000020-NV1-TST-M,10000020-NV1-TST-M,1.000000,3,3
10000022-TST-CP,10000022-TST-CP,1.000000,1,1
10000023-NV2-TST-CP,10000023-NV2-TST-CP,1.000000,7,7
10000027-M0-TST-M,10000027-M0-TST-M,1.000000,3,3
10000027-T0-TST-M,10000027-T0-TST-M,1.000000,3,3
10000027-T1-TST-M,10000027-T1-TST-M,1.000000,3,3


In [42]:
size_group = restricted_df.groupby('queried_record_id').size().reset_index().rename(columns={0:'size'})
queries_with_relevant_records = queries_with_relevant_records.merge(size_group, how='left', left_on='QueriedRecordID',right_on='queried_record_id')

In [44]:
numberOfPredictionsForRelevantRecords = queries_with_relevant_records.size.sum()

In [46]:
queries_with_relevant_records['r'] = queries_with_relevant_records["SelectedRecords"] / numberOfPredictionsForRelevantRecords

In [47]:
queries_with_relevant_records

,QueriedRecordID,Precision@K,SelectedRecords,AllRelevantRecords,queried_record_id,size,r
0,10000010-TST-CP,1.000000,1,1,10000010-TST-CP,1,0.000001
1,10000018-T4-TST-CP,1.000000,5,5,10000018-T4-TST-CP,5,0.000006
2,10000018-TST-M,1.000000,5,5,10000018-TST-M,5,0.000006
3,10000020-NV0-TST-CP,1.000000,3,3,10000020-NV0-TST-CP,3,0.000003
4,10000020-NV1-TST-M,1.000000,3,3,10000020-NV1-TST-M,3,0.000003
5,10000022-TST-CP,1.000000,1,1,10000022-TST-CP,1,0.000001
6,10000023-NV2-TST-CP,1.000000,7,7,10000023-NV2-TST-CP,7,0.000008
7,10000027-M0-TST-M,1.000000,3,3,10000027-M0-TST-M,3,0.000003
8,10000027-T0-TST-M,1.000000,3,3,10000027-T0-TST-M,3,0.000003
9,10000027-T1-TST-M,1.000000,3,3,10000027-T1-TST-M,3,0.000003


# F1-score

In [ ]:
# TODO calcolare l'F1 score, la formula è questa
#F1 = 2 * (precision * recall) / (precision + recall)

In [21]:
def compute_f1_score(precision, recall):
    F1 = 2 * (precision * recall) / (precision + recall)
    return F1

In [24]:
f1 = compute_f1_score(precision['AveragePrecision'], recall['AverageRecall'])
f1

0.6428389969768544

In [26]:
f1_rest = compute_f1_score(precision_rest['AveragePrecision'], recall_rest['AverageRecall'])
f1_rest

0.9870020809399241

## Cosa sbagliamo? Mettere nell'articolo ciò che sbagliamo in modo da far capire che in alcuni casi possiamo fare poco o nulla

In [100]:
train['phone'] = train.phone.astype(str)
test['phone'] = test.phone.astype(str)

In [10]:
def isin_function(pred, linked):
    res = np.empty(len(linked), dtype=int)
    for i in tqdm(range(len(linked))):
        res[i] = np.isin(linked[i], pred[i])
    return res

In [15]:
# Get only first 10 predictions
predicted_linked = s.ordered_linked.values
predicted_linked = [x[:10] for x in predicted_linked]

In [26]:
test = test.sort_values(by='record_id').reset_index(drop=True)

In [27]:
s['correct'] = isin_function(predicted_linked, test.linked_id.values) 

In [34]:
# do not consider queried records that are only in test
train['linked_id'] = train.linked_id.astype(int)
only_test = set(test.linked_id) - set(train.linked_id)
only_test_rec = test[test.linked_id.isin(only_test)].record_id.values

In [39]:
wrong_pred = s[~s.queried_record_id.isin(only_test_rec) & (s.correct == 0)]

In [136]:
wrong_pred_rec = iter(wrong_pred.queried_record_id.values)

In [185]:
curr = next(wrong_pred_rec)

In [186]:
test[test.record_id == curr]

,record_id,name,type,address,phone,email,modification,linked_id
16889,10024343-T2-TST-M,FAyHOy fOLDINGS LTD.,entity,NaN,3910990359.0,NaN,move row,10024343


In [187]:
#real 
l_test = test[test.record_id == curr].linked_id.values[0]
train[train.linked_id == l_test]

,record_id,name,type,address,phone,email,modification,linked_id
34427,10024343,FATHOM HOLDINGS LTD.,entity,NaN,3910990359.0,NaN,original,10024343
187108,10024343-M1,FATHOM HOLDINGS LTD.,entity,NaN,nan,NaN,missing,10024343
383407,10024343-T3,FATHOM HOLDINGStLTD.,entity,NaN,3910990359.0,NaN,typo,10024343
384044,10024343-T0,FATHOM HOLDINGS LTD.,entity,NaN,3910990359.0,NaN,typo,10024343
451269,10024343-T1,FATHOM HOLDINGS LTD.,entity,NaN,3910990359.0,NaN,typo,10024343


In [188]:
first_pred = s[s.queried_record_id == curr].ordered_record.values[0][0]

In [189]:
# predicted
train[train.record_id == first_pred]

,record_id,name,type,address,phone,email,modification,linked_id
666412,10179670-M0,YAP HOLDINGS LTD.,entity,NaN,nan,NaN,missing,10179670


In [190]:
s[s.queried_record_id == curr].ordered_scores.values[0][0]

-4.941296771187973

In [ ]:
# casi importanti: 10014653-T2-TST-M, 

In [111]:
a = s[s.queried_record_id == curr].ordered_record.values[0]

In [113]:
dfa = train[train.record_id.isin(a)]

In [114]:
dfa

,record_id,name,type,address,phone,email,modification,linked_id
24142,12202587-T2,CHU CkIxSHI,officer,RM 708; 4000 GUI GU DA JIE; CHANGSHAN NATIONAL...,nan,NaN,typo,12202587
44845,10056352-T1,aYAN INTERNATIONAL LIMITED,entity,NaN,420282152529.0,inquires@RYAN.INTERNATIONAL.LIMITED.gov,typo,10056352
58913,13004941-T1,LI Ckun Yui,officer,NOc 251; 2/F.; TAIPOTAU; TAI PO; NEW TERuITORI...,443855953745.0,ChunYui@xpkoud.gov,typo,13004941
62975,12162921,John Michael Gaynor,officer,Gable Cottage; 6 Bird's Hill Road; Oxsholt; Su...,15171670828.0,NaN,original,12162921
64762,10056352-NV2,RYAN INTERNATIONAL LIMITED,entity,NaN,420282152529.0,NaN,new_value,10056352
80078,10168656-T3,Calaff Cojp.,entity,NaN,425194609736.0,contakt@hotzqil.gov,typo,10168656
85185,10180512-NV0,ALKO IMPEX LTD.,entity,Commece House; Les Borghes; St. Peter Port; Gu...,420231528213.0,support@ALKO.IMPEX.LTD.ch,new_value,10180512
93876,12112459-T1,GARY DEMETRI,officer,30 Asricot Court; Melville; New York; USA,4116524402159.0,GARYDEMETRI@aol.ch,typo,12112459
99809,10082049-T2,CORMACK CkrPORATION,entity,NaN,17326225130.0,NaN,typo,10082049
101204,11006616-M1,BKS FAMILY OFFICE,intermediary,NaN,4150470218243.0,NaN,missing,11006616
